In [1]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

2023-04-16 09:37:46.485393: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:
traindata = pd.read_csv("./datasets/train.csv")
testdata = pd.read_csv("./datasets/test.csv")

In [25]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [40]:
traindata = traindata.drop(columns=['id'])
testdata = testdata.drop(columns=['id'])


In [41]:
from sklearn.preprocessing import LabelEncoder


In [42]:
train = traindata.select_dtypes(exclude=[np.number])
print(train.columns)
for feature in train.columns:
    traindata[feature] = LabelEncoder().fit_transform(traindata[feature])

Index(['proto', 'service', 'state', 'attack_cat'], dtype='object')


In [46]:
test = testdata.select_dtypes(exclude=[np.number])
print(test.columns)
for feature in test.columns:
    testdata[feature] = LabelEncoder().fit_transform(testdata[feature])

Index(['proto', 'service', 'state', 'attack_cat'], dtype='object')


In [66]:
traindata.to_csv('traindata.csv')

In [65]:
testdata.to_csv('testdata.csv')

In [56]:
traindata.iloc[:,43]


0        0
1        0
2        0
3        0
4        0
        ..
82327    0
82328    0
82329    0
82330    0
82331    0
Name: label, Length: 82332, dtype: int64

In [57]:
X = traindata.iloc[:,0:43]
Y = traindata.iloc[:,43]
C = testdata.iloc[:,43]
T = testdata.iloc[:,0:43]

In [58]:
trainX = np.array(X)
testT = np.array(T)

trainX.astype(float)
testT.astype(float)

array([[1.214780e-01, 1.130000e+02, 0.000000e+00, ..., 1.000000e+00,
        0.000000e+00, 6.000000e+00],
       [6.499020e-01, 1.130000e+02, 0.000000e+00, ..., 6.000000e+00,
        0.000000e+00, 6.000000e+00],
       [1.623129e+00, 1.130000e+02, 0.000000e+00, ..., 6.000000e+00,
        0.000000e+00, 6.000000e+00],
       ...,
       [9.000000e-06, 1.190000e+02, 2.000000e+00, ..., 1.200000e+01,
        0.000000e+00, 5.000000e+00],
       [9.000000e-06, 1.190000e+02, 2.000000e+00, ..., 3.000000e+01,
        0.000000e+00, 5.000000e+00],
       [9.000000e-06, 1.190000e+02, 2.000000e+00, ..., 3.000000e+01,
        0.000000e+00, 5.000000e+00]])

In [59]:
scaler = Normalizer().fit(trainX)
trainX = scaler.transform(trainX)

scaler = Normalizer().fit(testT)
testT = scaler.transform(testT)

y_train = np.array(Y)
y_test = np.array(C)


X_train = np.array(trainX)
X_test = np.array(testT)

In [60]:
batch_size = 64

In [ ]:
print("1 Layer DNN")

In [63]:
model = Sequential()
model.add(Dense(1024,input_dim=43,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [67]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn1layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn1layer/dnn1layer_model.hdf5")



Epoch 1/100
1280/1287 [============================>.] - ETA: 0s - loss: 0.4815 - accuracy: 0.7567
Epoch 1: loss improved from inf to 0.48153, saving model to kddresults/dnn1layer/checkpoint-01.hdf5
1287/1287 [==============================] - 11s 8ms/step - loss: 0.4815 - accuracy: 0.7567 - val_loss: 0.4073 - val_accuracy: 0.7468
Epoch 2/100
1274/1287 [============================>.] - ETA: 0s - loss: 0.4792 - accuracy: 0.7565
Epoch 2: loss improved from 0.48153 to 0.47940, saving model to kddresults/dnn1layer/checkpoint-02.hdf5
1287/1287 [==============================] - 11s 9ms/step - loss: 0.4794 - accuracy: 0.7561 - val_loss: 0.3978 - val_accuracy: 0.7849
Epoch 3/100
1280/1287 [============================>.] - ETA: 0s - loss: 0.4767 - accuracy: 0.7576
Epoch 3: loss improved from 0.47940 to 0.47686, saving model to kddresults/dnn1layer/checkpoint-03.hdf5
1287/1287 [==============================] - 12s 9ms/step - loss: 0.4769 - accuracy: 0.7575 - val_loss: 0.3921 - val_accuracy: 

Epoch 26/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4622 - accuracy: 0.7537
Epoch 26: loss did not improve from 0.46198
1287/1287 [==============================] - 9s 7ms/step - loss: 0.4622 - accuracy: 0.7537 - val_loss: 0.3946 - val_accuracy: 0.7469
Epoch 27/100
1273/1287 [============================>.] - ETA: 0s - loss: 0.4622 - accuracy: 0.7539
Epoch 27: loss improved from 0.46198 to 0.46192, saving model to kddresults/dnn1layer/checkpoint-27.hdf5
1287/1287 [==============================] - 9s 7ms/step - loss: 0.4619 - accuracy: 0.7541 - val_loss: 0.3554 - val_accuracy: 0.8220
Epoch 28/100
1277/1287 [============================>.] - ETA: 0s - loss: 0.4621 - accuracy: 0.7533
Epoch 28: loss improved from 0.46192 to 0.46173, saving model to kddresults/dnn1layer/checkpoint-28.hdf5
1287/1287 [==============================] - 9s 7ms/step - loss: 0.4617 - accuracy: 0.7534 - val_loss: 0.3647 - val_accuracy: 0.7953
Epoch 29/100
1275/1287 [=======================

1285/1287 [============================>.] - ETA: 0s - loss: 0.4610 - accuracy: 0.7518
Epoch 53: loss did not improve from 0.45995
1287/1287 [==============================] - 15s 11ms/step - loss: 0.4610 - accuracy: 0.7517 - val_loss: 0.3711 - val_accuracy: 0.7890
Epoch 54/100
1283/1287 [============================>.] - ETA: 0s - loss: 0.4598 - accuracy: 0.7526
Epoch 54: loss improved from 0.45995 to 0.45990, saving model to kddresults/dnn1layer/checkpoint-54.hdf5
1287/1287 [==============================] - 14s 11ms/step - loss: 0.4599 - accuracy: 0.7525 - val_loss: 0.3778 - val_accuracy: 0.7535
Epoch 55/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4603 - accuracy: 0.7523
Epoch 55: loss did not improve from 0.45990
1287/1287 [==============================] - 14s 11ms/step - loss: 0.4603 - accuracy: 0.7523 - val_loss: 0.3901 - val_accuracy: 0.7557
Epoch 56/100
1278/1287 [============================>.] - ETA: 0s - loss: 0.4594 - accuracy: 0.7533
Epoch 56: loss 

Epoch 80/100
1280/1287 [============================>.] - ETA: 0s - loss: 0.4553 - accuracy: 0.7536
Epoch 80: loss improved from 0.45545 to 0.45533, saving model to kddresults/dnn1layer/checkpoint-80.hdf5
1287/1287 [==============================] - 9s 7ms/step - loss: 0.4553 - accuracy: 0.7536 - val_loss: 0.3802 - val_accuracy: 0.7504
Epoch 81/100
1278/1287 [============================>.] - ETA: 0s - loss: 0.4552 - accuracy: 0.7528
Epoch 81: loss improved from 0.45533 to 0.45518, saving model to kddresults/dnn1layer/checkpoint-81.hdf5
1287/1287 [==============================] - 9s 7ms/step - loss: 0.4552 - accuracy: 0.7528 - val_loss: 0.3602 - val_accuracy: 0.7941
Epoch 82/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4546 - accuracy: 0.7524
Epoch 82: loss improved from 0.45518 to 0.45464, saving model to kddresults/dnn1layer/checkpoint-82.hdf5
1287/1287 [==============================] - 10s 8ms/step - loss: 0.4546 - accuracy: 0.7524 - val_loss: 0.3810 - val_ac

In [68]:
print("2 Layer DNN")

2 Layer DNN


In [69]:
model = Sequential()
model.add(Dense(1024,input_dim=43,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [71]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn2layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn2layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn2layer/dnn2layer_model.hdf5")

Epoch 1/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4512 - accuracy: 0.7636
Epoch 1: loss improved from inf to 0.45123, saving model to kddresults/dnn2layer/checkpoint-01.hdf5
1287/1287 [==============================] - 36s 28ms/step - loss: 0.4512 - accuracy: 0.7636 - val_loss: 0.3732 - val_accuracy: 0.8119
Epoch 2/100
1284/1287 [============================>.] - ETA: 0s - loss: 0.4512 - accuracy: 0.7612
Epoch 2: loss did not improve from 0.45123
1287/1287 [==============================] - 35s 27ms/step - loss: 0.4512 - accuracy: 0.7612 - val_loss: 0.3905 - val_accuracy: 0.7804
Epoch 3/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4508 - accuracy: 0.7627
Epoch 3: loss improved from 0.45123 to 0.45079, saving model to kddresults/dnn2layer/checkpoint-03.hdf5
1287/1287 [==============================] - 30s 23ms/step - loss: 0.4508 - accuracy: 0.7627 - val_loss: 0.3807 - val_accuracy: 0.7477
Epoch 4/100
1285/1287 [============================

Epoch 29/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4493 - accuracy: 0.7624
Epoch 29: loss improved from 0.44944 to 0.44932, saving model to kddresults/dnn2layer/checkpoint-29.hdf5
1287/1287 [==============================] - 50s 39ms/step - loss: 0.4493 - accuracy: 0.7624 - val_loss: 0.3629 - val_accuracy: 0.8132
Epoch 30/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4504 - accuracy: 0.7632
Epoch 30: loss did not improve from 0.44932
1287/1287 [==============================] - 41s 32ms/step - loss: 0.4504 - accuracy: 0.7632 - val_loss: 0.3643 - val_accuracy: 0.7622
Epoch 31/100
1284/1287 [============================>.] - ETA: 0s - loss: 0.4496 - accuracy: 0.7625
Epoch 31: loss did not improve from 0.44932
1287/1287 [==============================] - 43s 33ms/step - loss: 0.4496 - accuracy: 0.7625 - val_loss: 0.3598 - val_accuracy: 0.8124
Epoch 32/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4497 - accuracy: 0.7623
Ep

1285/1287 [============================>.] - ETA: 0s - loss: 0.4492 - accuracy: 0.7632
Epoch 57: loss did not improve from 0.44851
1287/1287 [==============================] - 32s 25ms/step - loss: 0.4492 - accuracy: 0.7631 - val_loss: 0.3775 - val_accuracy: 0.7487
Epoch 58/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4492 - accuracy: 0.7631
Epoch 58: loss did not improve from 0.44851
1287/1287 [==============================] - 31s 24ms/step - loss: 0.4492 - accuracy: 0.7631 - val_loss: 0.3732 - val_accuracy: 0.7944
Epoch 59/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4490 - accuracy: 0.7627
Epoch 59: loss did not improve from 0.44851
1287/1287 [==============================] - 34s 27ms/step - loss: 0.4490 - accuracy: 0.7626 - val_loss: 0.3680 - val_accuracy: 0.8152
Epoch 60/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4494 - accuracy: 0.7629
Epoch 60: loss did not improve from 0.44851
1287/1287 [=====================

Epoch 86/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4491 - accuracy: 0.7627
Epoch 86: loss did not improve from 0.44805
1287/1287 [==============================] - 33s 26ms/step - loss: 0.4491 - accuracy: 0.7627 - val_loss: 0.3656 - val_accuracy: 0.7637
Epoch 87/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4486 - accuracy: 0.7633
Epoch 87: loss did not improve from 0.44805
1287/1287 [==============================] - 37s 28ms/step - loss: 0.4486 - accuracy: 0.7633 - val_loss: 0.3590 - val_accuracy: 0.8078
Epoch 88/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4482 - accuracy: 0.7623
Epoch 88: loss did not improve from 0.44805
1287/1287 [==============================] - 37s 29ms/step - loss: 0.4482 - accuracy: 0.7623 - val_loss: 0.3612 - val_accuracy: 0.8000
Epoch 89/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4486 - accuracy: 0.7624
Epoch 89: loss did not improve from 0.44805
1287/1287 [========

In [72]:
print("3 Layer DNN")

3 Layer DNN


In [73]:
model = Sequential()
model.add(Dense(1024,input_dim=43,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [74]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn3layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn3layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn3layer/dnn3layer_model.hdf5")

Epoch 1/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.5060 - accuracy: 0.7550
Epoch 1: loss improved from inf to 0.50598, saving model to kddresults/dnn3layer/checkpoint-01.hdf5
1287/1287 [==============================] - 43s 32ms/step - loss: 0.5060 - accuracy: 0.7550 - val_loss: 0.4343 - val_accuracy: 0.7452
Epoch 2/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4899 - accuracy: 0.7574
Epoch 2: loss improved from 0.50598 to 0.48990, saving model to kddresults/dnn3layer/checkpoint-02.hdf5
1287/1287 [==============================] - 47s 36ms/step - loss: 0.4899 - accuracy: 0.7573 - val_loss: 0.4037 - val_accuracy: 0.7657
Epoch 3/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4825 - accuracy: 0.7588
Epoch 3: loss improved from 0.48990 to 0.48248, saving model to kddresults/dnn3layer/checkpoint-03.hdf5
1287/1287 [==============================] - 57s 44ms/step - loss: 0.4825 - accuracy: 0.7588 - val_loss: 0.4189 - val_accurac

1287/1287 [==============================] - 45s 35ms/step - loss: 0.4535 - accuracy: 0.7621 - val_loss: 0.3674 - val_accuracy: 0.7976
Epoch 27/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4529 - accuracy: 0.7610
Epoch 27: loss improved from 0.45347 to 0.45284, saving model to kddresults/dnn3layer/checkpoint-27.hdf5
1287/1287 [==============================] - 43s 33ms/step - loss: 0.4528 - accuracy: 0.7610 - val_loss: 0.3671 - val_accuracy: 0.8087
Epoch 28/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4529 - accuracy: 0.7622
Epoch 28: loss did not improve from 0.45284
1287/1287 [==============================] - 48s 37ms/step - loss: 0.4529 - accuracy: 0.7622 - val_loss: 0.3564 - val_accuracy: 0.8020
Epoch 29/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4536 - accuracy: 0.7607
Epoch 29: loss did not improve from 0.45284
1287/1287 [==============================] - 46s 36ms/step - loss: 0.4536 - accuracy: 0.7607 - val_los

1286/1287 [============================>.] - ETA: 0s - loss: 0.4514 - accuracy: 0.7614
Epoch 54: loss did not improve from 0.45121
1287/1287 [==============================] - 59s 46ms/step - loss: 0.4514 - accuracy: 0.7614 - val_loss: 0.3581 - val_accuracy: 0.7505
Epoch 55/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4509 - accuracy: 0.7609
Epoch 55: loss improved from 0.45121 to 0.45095, saving model to kddresults/dnn3layer/checkpoint-55.hdf5
1287/1287 [==============================] - 52s 40ms/step - loss: 0.4510 - accuracy: 0.7610 - val_loss: 0.3661 - val_accuracy: 0.7866
Epoch 56/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4511 - accuracy: 0.7621
Epoch 56: loss did not improve from 0.45095
1287/1287 [==============================] - 53s 42ms/step - loss: 0.4511 - accuracy: 0.7621 - val_loss: 0.3615 - val_accuracy: 0.7978
Epoch 57/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4505 - accuracy: 0.7621
Epoch 57: loss 

1286/1287 [============================>.] - ETA: 0s - loss: 0.4501 - accuracy: 0.7626
Epoch 82: loss did not improve from 0.44974
1287/1287 [==============================] - 43s 33ms/step - loss: 0.4500 - accuracy: 0.7627 - val_loss: 0.3588 - val_accuracy: 0.8074
Epoch 83/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4501 - accuracy: 0.7616
Epoch 83: loss did not improve from 0.44974
1287/1287 [==============================] - 49s 38ms/step - loss: 0.4501 - accuracy: 0.7616 - val_loss: 0.3607 - val_accuracy: 0.8070
Epoch 84/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4501 - accuracy: 0.7606
Epoch 84: loss did not improve from 0.44974
1287/1287 [==============================] - 48s 38ms/step - loss: 0.4501 - accuracy: 0.7606 - val_loss: 0.3611 - val_accuracy: 0.8031
Epoch 85/100
1285/1287 [============================>.] - ETA: 0s - loss: 0.4492 - accuracy: 0.7618
Epoch 85: loss improved from 0.44974 to 0.44927, saving model to kddresults/

In [75]:
print("4 Layer DNN")

4 Layer DNN


In [76]:
model = Sequential()
model.add(Dense(1024,input_dim=43,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [77]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn4layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn4layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn4layer/dnn4layer_model.hdf5")

Epoch 1/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.5067 - accuracy: 0.7546
Epoch 1: loss improved from inf to 0.50662, saving model to kddresults/dnn4layer/checkpoint-01.hdf5
1287/1287 [==============================] - 72s 55ms/step - loss: 0.5066 - accuracy: 0.7546 - val_loss: 0.4191 - val_accuracy: 0.7457
Epoch 2/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4863 - accuracy: 0.7577
Epoch 2: loss improved from 0.50662 to 0.48625, saving model to kddresults/dnn4layer/checkpoint-02.hdf5
1287/1287 [==============================] - 66s 51ms/step - loss: 0.4863 - accuracy: 0.7577 - val_loss: 0.3970 - val_accuracy: 0.7897
Epoch 3/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4801 - accuracy: 0.7594
Epoch 3: loss improved from 0.48625 to 0.48006, saving model to kddresults/dnn4layer/checkpoint-03.hdf5
1287/1287 [==============================] - 67s 52ms/step - loss: 0.4801 - accuracy: 0.7594 - val_loss: 0.3932 - val_accurac

Epoch 26/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4577 - accuracy: 0.7593
Epoch 26: loss improved from 0.45836 to 0.45772, saving model to kddresults/dnn4layer/checkpoint-26.hdf5
1287/1287 [==============================] - 144s 112ms/step - loss: 0.4577 - accuracy: 0.7593 - val_loss: 0.3635 - val_accuracy: 0.8117
Epoch 27/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4568 - accuracy: 0.7608
Epoch 27: loss improved from 0.45772 to 0.45674, saving model to kddresults/dnn4layer/checkpoint-27.hdf5
1287/1287 [==============================] - 119s 92ms/step - loss: 0.4567 - accuracy: 0.7608 - val_loss: 0.3797 - val_accuracy: 0.7470
Epoch 28/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4567 - accuracy: 0.7611
Epoch 28: loss improved from 0.45674 to 0.45669, saving model to kddresults/dnn4layer/checkpoint-28.hdf5
1287/1287 [==============================] - 108s 84ms/step - loss: 0.4567 - accuracy: 0.7611 - val_loss: 0.3865

1287/1287 [==============================] - 64s 50ms/step - loss: 0.4524 - accuracy: 0.7614 - val_loss: 0.3551 - val_accuracy: 0.8089
Epoch 53/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4516 - accuracy: 0.7615
Epoch 53: loss improved from 0.45196 to 0.45169, saving model to kddresults/dnn4layer/checkpoint-53.hdf5
1287/1287 [==============================] - 60s 47ms/step - loss: 0.4517 - accuracy: 0.7615 - val_loss: 0.3480 - val_accuracy: 0.8116
Epoch 54/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4529 - accuracy: 0.7616
Epoch 54: loss did not improve from 0.45169
1287/1287 [==============================] - 59s 46ms/step - loss: 0.4529 - accuracy: 0.7616 - val_loss: 0.3597 - val_accuracy: 0.7975
Epoch 55/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4518 - accuracy: 0.7617
Epoch 55: loss did not improve from 0.45169
1287/1287 [==============================] - 59s 46ms/step - loss: 0.4519 - accuracy: 0.7616 - val_los

1287/1287 [==============================] - ETA: 0s - loss: 0.4506 - accuracy: 0.7623
Epoch 80: loss did not improve from 0.44967
1287/1287 [==============================] - 51s 39ms/step - loss: 0.4506 - accuracy: 0.7623 - val_loss: 0.3626 - val_accuracy: 0.7926
Epoch 81/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4501 - accuracy: 0.7608
Epoch 81: loss did not improve from 0.44967
1287/1287 [==============================] - 50s 39ms/step - loss: 0.4501 - accuracy: 0.7608 - val_loss: 0.3591 - val_accuracy: 0.8096
Epoch 82/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4502 - accuracy: 0.7626
Epoch 82: loss did not improve from 0.44967
1287/1287 [==============================] - 50s 39ms/step - loss: 0.4502 - accuracy: 0.7626 - val_loss: 0.3525 - val_accuracy: 0.8072
Epoch 83/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4503 - accuracy: 0.7629
Epoch 83: loss did not improve from 0.44967
1287/1287 [=====================

In [78]:
print("5 Layer DNN")

5 Layer DNN


In [79]:
model = Sequential()
model.add(Dense(1024,input_dim=43,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [80]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn5layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn5layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn5layer/dnn5layer_model.hdf5")

Epoch 1/100
1287/1287 [==============================] - ETA: 0s - loss: 0.5085 - accuracy: 0.7548
Epoch 1: loss improved from inf to 0.50853, saving model to kddresults/dnn5layer/checkpoint-01.hdf5
1287/1287 [==============================] - 54s 41ms/step - loss: 0.5085 - accuracy: 0.7548 - val_loss: 0.4589 - val_accuracy: 0.7451
Epoch 2/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4916 - accuracy: 0.7560
Epoch 2: loss improved from 0.50853 to 0.49159, saving model to kddresults/dnn5layer/checkpoint-02.hdf5
1287/1287 [==============================] - 53s 41ms/step - loss: 0.4916 - accuracy: 0.7560 - val_loss: 0.4191 - val_accuracy: 0.7623
Epoch 3/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4821 - accuracy: 0.7594
Epoch 3: loss improved from 0.49159 to 0.48215, saving model to kddresults/dnn5layer/checkpoint-03.hdf5
1287/1287 [==============================] - 52s 41ms/step - loss: 0.4821 - accuracy: 0.7594 - val_loss: 0.3946 - val_accurac

1286/1287 [============================>.] - ETA: 0s - loss: 0.4552 - accuracy: 0.7607
Epoch 26: loss improved from 0.45634 to 0.45515, saving model to kddresults/dnn5layer/checkpoint-26.hdf5
1287/1287 [==============================] - 52s 41ms/step - loss: 0.4552 - accuracy: 0.7607 - val_loss: 0.3810 - val_accuracy: 0.8057
Epoch 27/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4554 - accuracy: 0.7603
Epoch 27: loss did not improve from 0.45515
1287/1287 [==============================] - 51s 40ms/step - loss: 0.4554 - accuracy: 0.7603 - val_loss: 0.3871 - val_accuracy: 0.7725
Epoch 28/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4569 - accuracy: 0.7597
Epoch 28: loss did not improve from 0.45515
1287/1287 [==============================] - 51s 40ms/step - loss: 0.4569 - accuracy: 0.7597 - val_loss: 0.3880 - val_accuracy: 0.8059
Epoch 29/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4552 - accuracy: 0.7609
Epoch 29: loss 

Epoch 54/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4526 - accuracy: 0.7610
Epoch 54: loss did not improve from 0.45138
1287/1287 [==============================] - 63s 49ms/step - loss: 0.4526 - accuracy: 0.7610 - val_loss: 0.4378 - val_accuracy: 0.7566
Epoch 55/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4526 - accuracy: 0.7619
Epoch 55: loss did not improve from 0.45138
1287/1287 [==============================] - 98s 76ms/step - loss: 0.4526 - accuracy: 0.7619 - val_loss: 0.4369 - val_accuracy: 0.7823
Epoch 56/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4525 - accuracy: 0.7617
Epoch 56: loss did not improve from 0.45138
1287/1287 [==============================] - 60s 46ms/step - loss: 0.4525 - accuracy: 0.7617 - val_loss: 0.4269 - val_accuracy: 0.8031
Epoch 57/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4537 - accuracy: 0.7617
Epoch 57: loss did not improve from 0.45138
1287/1287 [========

1287/1287 [==============================] - 56s 43ms/step - loss: 0.4516 - accuracy: 0.7621 - val_loss: 0.3946 - val_accuracy: 0.7730
Epoch 83/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4506 - accuracy: 0.7624
Epoch 83: loss did not improve from 0.44979
1287/1287 [==============================] - 56s 44ms/step - loss: 0.4506 - accuracy: 0.7624 - val_loss: 0.3689 - val_accuracy: 0.8024
Epoch 84/100
1286/1287 [============================>.] - ETA: 0s - loss: 0.4502 - accuracy: 0.7626
Epoch 84: loss did not improve from 0.44979
1287/1287 [==============================] - 59s 46ms/step - loss: 0.4502 - accuracy: 0.7627 - val_loss: 0.3813 - val_accuracy: 0.7931
Epoch 85/100
1287/1287 [==============================] - ETA: 0s - loss: 0.4508 - accuracy: 0.7618
Epoch 85: loss did not improve from 0.44979
1287/1287 [==============================] - 66s 51ms/step - loss: 0.4508 - accuracy: 0.7618 - val_loss: 0.3830 - val_accuracy: 0.8090
Epoch 86/100
1287/1287 [====

In [81]:
print("******************")

******************
